In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
data_path_and_labels = pd.read_csv("../mapped_data/ant-1.7.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [29]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [30]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent'

In [31]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [32]:
d = pd.read_csv("../mapped_data/ant-1.7.csv")
d['data_path'][0]

'dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [33]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [34]:
d

,data_path,bugs
0,../dataset/promise/defect data/ant-1.7/apache-...,1
1,../dataset/promise/defect data/ant-1.7/apache-...,0
2,../dataset/promise/defect data/ant-1.7/apache-...,1
3,../dataset/promise/defect data/ant-1.7/apache-...,0
4,../dataset/promise/defect data/ant-1.7/apache-...,7
...,...,...
736,../dataset/promise/defect data/ant-1.7/apache-...,0
737,../dataset/promise/defect data/ant-1.7/apache-...,0
738,../dataset/promise/defect data/ant-1.7/apache-...,0
739,../dataset/promise/defect data/ant-1.7/apache-...,6


In [35]:
train, test = train_test_split(d, test_size=0.2)

In [36]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [37]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [38]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

10670.0

In [39]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [40]:
train_y

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  2,  0,  7,  0,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,
        3,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  4,  0,  0,  0,  1,  6,  0,  0,  0,  1,  0,  0,
        3,  0,  0,  0,  0,  8,  1,  0,  0,  2,  0,  0,  0,  0,  1,  1,  0,
        0,  4,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,
        0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  4,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,
        2,  0,  2,  0,  1

In [41]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [42]:
train_x.shape

(592, 3378)

In [43]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [44]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
19/19 [==============================] - 1s 25ms/step - loss: 1.4711 - mse: 1.4711 - mae: 0.4443 - root_mean_squared_error: 1.2129 - mean_squared_logarithmic_error: 0.2610
Epoch 2/100
19/19 [==============================] - 0s 23ms/step - loss: 1.2565 - mse: 1.2565 - mae: 0.6651 - root_mean_squared_error: 1.1209 - mean_squared_logarithmic_error: 0.2261
Epoch 3/100
19/19 [==============================] - 0s 20ms/step - loss: 1.0047 - mse: 1.0047 - mae: 0.5473 - root_mean_squared_error: 1.0024 - mean_squared_logarithmic_error: 0.1593
Epoch 4/100
19/19 [==============================] - 0s 22ms/step - loss: 0.8579 - mse: 0.8579 - mae: 0.5181 - root_mean_squared_error: 0.9262 - mean_squared_logarithmic_error: 0.1470
Epoch 5/100
19/19 [==============================] - 0s 20ms/step - loss: 0.7751 - mse: 0.7751 - mae: 0.4774 - root_mean_squared_error: 0.8804 - mean_squared_logarithmic_error: 0.1420
Epoch 6/100
19/19 [==============================] - 0s 20ms/step - loss: 0.7311

19/19 [==============================] - 0s 20ms/step - loss: 0.0761 - mse: 0.0761 - mae: 0.1229 - root_mean_squared_error: 0.2758 - mean_squared_logarithmic_error: 0.0224
Epoch 45/100
19/19 [==============================] - 0s 25ms/step - loss: 0.0716 - mse: 0.0716 - mae: 0.1178 - root_mean_squared_error: 0.2676 - mean_squared_logarithmic_error: 0.0213
Epoch 46/100
19/19 [==============================] - 0s 22ms/step - loss: 0.0689 - mse: 0.0689 - mae: 0.1146 - root_mean_squared_error: 0.2625 - mean_squared_logarithmic_error: 0.0205
Epoch 47/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0639 - mse: 0.0639 - mae: 0.1157 - root_mean_squared_error: 0.2529 - mean_squared_logarithmic_error: 0.0201
Epoch 48/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0615 - mse: 0.0615 - mae: 0.1045 - root_mean_squared_error: 0.2481 - mean_squared_logarithmic_error: 0.0182
Epoch 49/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0568 - mse:

19/19 [==============================] - 0s 21ms/step - loss: 0.0131 - mse: 0.0131 - mae: 0.0338 - root_mean_squared_error: 0.1144 - mean_squared_logarithmic_error: 0.0050
Epoch 89/100
19/19 [==============================] - 0s 20ms/step - loss: 0.0124 - mse: 0.0124 - mae: 0.0301 - root_mean_squared_error: 0.1115 - mean_squared_logarithmic_error: 0.0048
Epoch 90/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0122 - mse: 0.0122 - mae: 0.0281 - root_mean_squared_error: 0.1105 - mean_squared_logarithmic_error: 0.0045
Epoch 91/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0120 - mse: 0.0120 - mae: 0.0287 - root_mean_squared_error: 0.1097 - mean_squared_logarithmic_error: 0.0046
Epoch 92/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0116 - mse: 0.0116 - mae: 0.0282 - root_mean_squared_error: 0.1075 - mean_squared_logarithmic_error: 0.0044
Epoch 93/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0114 - mse:

In [45]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:18:02 ago. (Use '!kill 21467' to kill it.)

In [46]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 6ms/step - loss: 0.9885 - mse: 0.9885 - mae: 0.4856 - root_mean_squared_error: 0.9943 - mean_squared_logarithmic_error: 0.1551


[0.9885339736938477,
 0.9885339736938477,
 0.4855883717536926,
 0.994250476360321,
 0.15511944890022278]

In [10]:
data_path_and_labels = pd.read_csv("../mapped_data/ant-1.7.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [11]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [13]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [14]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
0,2480.0,6397.0,10419.0,2480.0,10419.0,6397.0,10419.0,10419.0,2480.0,3202.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2470.0,5355.0,2480.0,5455.0,2480.0,5065.0,2480.0,5453.0,3200.0,5063.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2338.0,2480.0,6072.0,9002.0,6072.0,2480.0,6072.0,6072.0,5355.0,2480.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2333.0,6397.0,1709.0,6397.0,1348.0,6397.0,874.0,6397.0,574.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2471.0,1123.0,4331.0,1123.0,1123.0,1123.0,1672.0,6938.0,6074.0,6938.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,2087.0,231.0,8254.0,2107.0,8494.0,2107.0,2087.0,6253.0,1196.0,8254.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,356.0,6397.0,3665.0,356.0,6356.0,6356.0,5937.0,5318.0,6397.0,8513.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,197.0,6397.0,3038.0,6397.0,1426.0,6397.0,1430.0,6397.0,1866.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,253.0,6397.0,6397.0,1427.0,6397.0,1959.0,6397.0,276.0,1959.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
0,1,2480.0,6397.0,10419.0,2480.0,10419.0,6397.0,10419.0,10419.0,2480.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2470.0,5355.0,2480.0,5455.0,2480.0,5065.0,2480.0,5453.0,3200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2338.0,2480.0,6072.0,9002.0,6072.0,2480.0,6072.0,6072.0,5355.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2333.0,6397.0,1709.0,6397.0,1348.0,6397.0,874.0,6397.0,574.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,2471.0,1123.0,4331.0,1123.0,1123.0,1123.0,1672.0,6938.0,6074.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0,2087.0,231.0,8254.0,2107.0,8494.0,2107.0,2087.0,6253.0,1196.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,0,356.0,6397.0,3665.0,356.0,6356.0,6356.0,5937.0,5318.0,6397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,0,197.0,6397.0,3038.0,6397.0,1426.0,6397.0,1430.0,6397.0,1866.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,6,253.0,6397.0,6397.0,1427.0,6397.0,1959.0,6397.0,276.0,1959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
train, test = train_test_split(data, test_size=0.2)

In [17]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
506,0,1879.0,2106.0,8309.0,1794.0,8098.0,1879.0,1879.0,6175.0,2106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
420,0,2316.0,986.0,4053.0,986.0,4052.0,2313.0,10334.0,9339.0,10334.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192,1,1013.0,881.0,5303.0,577.0,577.0,2106.0,880.0,2106.0,2106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0,2447.0,6897.0,7284.0,2106.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,0,692.0,690.0,693.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0,415.0,2106.0,6966.0,2106.0,7294.0,2106.0,7265.0,6397.0,3111.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512,0,1751.0,9176.0,7827.0,1749.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,0,4453.0,6897.0,5824.0,4453.0,797.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
331,0,1202.0,2106.0,6401.0,2106.0,6402.0,2106.0,6403.0,6397.0,3395.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377
394,0,1483.0,3158.0,426.0,426.0,3509.0,426.0,6187.0,5909.0,3098.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
724,0,1275.0,1275.0,9722.0,2106.0,6366.0,2106.0,2898.0,2106.0,7686.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
583,0,1627.0,1627.0,10182.0,2106.0,2107.0,3145.0,2107.0,6187.0,6029.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0,837.0,1007.0,1751.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0,2210.0,2205.0,9824.0,2106.0,9867.0,2106.0,9872.0,1938.0,10564.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,363.0,2106.0,3864.0,986.0,7886.0,2106.0,7892.0,3317.0,9409.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
418,0,986.0,6786.0,2377.0,2377.0,8960.0,4635.0,986.0,6786.0,4635.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0,1302.0,6397.0,1468.0,4453.0,3111.0,6897.0,1479.0,426.0,3463.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284,0,2106.0,7347.0,3111.0,2553.0,3111.0,3964.0,3111.0,9445.0,6429.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [20]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [21]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [22]:
X_train = train.values
X_test = test.values

In [23]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [24]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [25]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
19/19 [==============================] - 2s 10ms/step - loss: 1.2578 - mse: 1.2578 - mae: 0.4913 - root_mean_squared_error: 1.1215 - mean_squared_logarithmic_error: 0.2242
Epoch 2/100
19/19 [==============================] - 0s 11ms/step - loss: 1.1572 - mse: 1.1572 - mae: 0.4729 - root_mean_squared_error: 1.0757 - mean_squared_logarithmic_error: 0.2048
Epoch 3/100
19/19 [==============================] - 0s 13ms/step - loss: 0.9541 - mse: 0.9541 - mae: 0.4503 - root_mean_squared_error: 0.9768 - mean_squared_logarithmic_error: 0.1752
Epoch 4/100
19/19 [==============================] - 0s 12ms/step - loss: 0.9835 - mse: 0.9835 - mae: 0.4663 - root_mean_squared_error: 0.9917 - mean_squared_logarithmic_error: 0.1747
Epoch 5/100
19/19 [==============================] - 0s 12ms/step - loss: 1.0011 - mse: 1.0011 - mae: 0.4637 - root_mean_squared_error: 1.0006 - mean_squared_logarithmic_error: 0.1856
Epoch 6/100
19/19 [==============================] - 0s 13ms/step - loss: 1.0378

19/19 [==============================] - 0s 12ms/step - loss: 0.8982 - mse: 0.8982 - mae: 0.4704 - root_mean_squared_error: 0.9477 - mean_squared_logarithmic_error: 0.1646
Epoch 46/100
19/19 [==============================] - 0s 12ms/step - loss: 0.9167 - mse: 0.9167 - mae: 0.4492 - root_mean_squared_error: 0.9574 - mean_squared_logarithmic_error: 0.1640
Epoch 47/100
19/19 [==============================] - 0s 13ms/step - loss: 0.9098 - mse: 0.9098 - mae: 0.4716 - root_mean_squared_error: 0.9538 - mean_squared_logarithmic_error: 0.1798
Epoch 48/100
19/19 [==============================] - 0s 12ms/step - loss: 0.8618 - mse: 0.8618 - mae: 0.4281 - root_mean_squared_error: 0.9283 - mean_squared_logarithmic_error: 0.1545
Epoch 49/100
19/19 [==============================] - 0s 12ms/step - loss: 0.9068 - mse: 0.9068 - mae: 0.4645 - root_mean_squared_error: 0.9522 - mean_squared_logarithmic_error: 0.1687
Epoch 50/100
19/19 [==============================] - 0s 13ms/step - loss: 0.9212 - mse:

19/19 [==============================] - 0s 11ms/step - loss: 0.8425 - mse: 0.8425 - mae: 0.4685 - root_mean_squared_error: 0.9179 - mean_squared_logarithmic_error: 0.1550
Epoch 90/100
19/19 [==============================] - 0s 12ms/step - loss: 0.8709 - mse: 0.8709 - mae: 0.4745 - root_mean_squared_error: 0.9332 - mean_squared_logarithmic_error: 0.1595
Epoch 91/100
19/19 [==============================] - 0s 13ms/step - loss: 0.8385 - mse: 0.8385 - mae: 0.4755 - root_mean_squared_error: 0.9157 - mean_squared_logarithmic_error: 0.1554
Epoch 92/100
19/19 [==============================] - 0s 12ms/step - loss: 0.9471 - mse: 0.9471 - mae: 0.5038 - root_mean_squared_error: 0.9732 - mean_squared_logarithmic_error: 0.1676
Epoch 93/100
19/19 [==============================] - 0s 13ms/step - loss: 0.8862 - mse: 0.8862 - mae: 0.4961 - root_mean_squared_error: 0.9414 - mean_squared_logarithmic_error: 0.1652
Epoch 94/100
19/19 [==============================] - 0s 13ms/step - loss: 0.8783 - mse:

In [26]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 4ms/step - loss: 0.8999 - mse: 0.8999 - mae: 0.4634 - root_mean_squared_error: 0.9486 - mean_squared_logarithmic_error: 0.1603


[0.8999199867248535,
 0.8999199867248535,
 0.4634470045566559,
 0.9486411213874817,
 0.16034455597400665]